loading packages

In [ ]:
using Revise, juobs, BDIO, ADerrors, PyPlot, LaTeXStrings

paths to mesons.dat, ms1.dat


In [ ]:
const path_J303 = "/Users/ale/Desktop/data/J303/J303r003_k0.1372131_mup0.001512_mup0.00957_mup0.13300_mup0.14000_mup0.14700.mesons.dat"  
const path_J303rwf = "/Users/ale/Desktop/data/rwf/J303r003.ms1.dat"

const path_N200r000 = "/Users/ale/Desktop/data/N200/N200r000_k0.1373124_mup0.00242_mup0.010777_mup0.172805_mup0.18190_mup0.190995.mesons.dat"
const path_N200r001 = "/Users/ale/Desktop/data/N200/N200r001_k0.1373124_mup0.00242_mup0.010777_mup0.172805_mup0.18190_mup0.190995.mesons.dat"
const path_N200r000_rwf = "/Users/ale/Desktop/data/rwf/N200r000.ms1.dat"
const path_N200r001_rwf = "/Users/ale/Desktop/data/rwf/N200r001.ms1.dat"



Loading data and create primary observables 

In [ ]:
## single replica with rw factor (J303)
data = read_mesons(path_J303, "G5", "G5")                        ## read from path_J303 the pp correlator. Return a vector of CData structures
rwf_J303 = read_ms1(path_J303rwf)                                ## read from path_J303rwf the rw factors. Return a matrix W[irw, icfg]
J303_pp = corr_obs.(data, rw=rwf_J303, L=64)                     ## create primary obs. reweighted. Creates a Corr struct with the given CData struct 

## dealing with multiple replicas  with rw factors(N200)
data1 = read_mesons([path_N200r000, path_N200r001], "G5", "G5")   ## multiple dispatch in action: same name, different arguments

rwf_N200r000 = read_ms1(path_N200r000_rwf)
rwf_N200r001 = read_ms1(path_N200r001_rwf)

@time N200_pp = corr_obs.(data1, L=48, rw=[rwf_N200r000, rwf_N200r001])  ##use the @time macro to get timing info

In [ ]:
## exploring the juobs.Corr struct
## the observable is accessed with J303_pp[i].obs
for i=1:length(J303_pp)
    println("i = ", i, L" $\mu$ values: ", J303_pp[i].mu, "                gamma structure: ", J303_pp[i].gamma , "            source: ", J303_pp[i].y0)
end

computing effective masses 

In [ ]:
plat = [[110,160] for i=1:length(J303_pp)] ## define an array of plateau for each combination of \mu (J303)
plat1 = [[85,115] for i=1:length(N200_pp)] ## define an array of plateau for each combination of \mu (N200)
m_J303 = meff.(J303_pp, plat, pl=true)    ## compute effective masses. The . notation applies the function meff to all the elements of the J303_pp (N200_pp) array of correlators
m_N200 = meff.(N200_pp, plat1, pl=false)

uwerr.(m_N200)
uwerr.(m_J303)

println("The light-light effective  mass is:\n", " N200: ", m_N200[1], " \n J303: ", m_J303[1] )

Decay constants extraction 


In [ ]:
plat = [[110,160] for i=1:length(J303_pp)] ## define an array of plateaus for each combination of \mu (J303)
plat1 = [[80,110] for i=1:length(N200_pp)] ## define an array of plateaus for each combination of \mu (N200)

#mu_list_J303 = getfield.(J303_pp, :mu)     ## array with all possible values of \mu
#mu_list_N200 = getfield.(N200_pp, :mu)     ## array with all possible values of \mu

f_J303 = dec_const_pcvc.(J303_pp, plat, m_J303, pl=false )  ## array of decay constants for J303
f_N200 = dec_const_pcvc.(N200_pp, plat1, m_N200, pl=false )  ## array of decay constants for N200
println("The light-light decay constant is:\n", " N200: ", f_N200[1], "\n J303: ", f_J303[1] )

Constants are stored in the juobs_const file. The t_not values are taken from the scale setting paper by M.Bruno et al.
Here we load such file and we express masses in terms of sqrt(8 t_0)

In [ ]:
include("/Users/ale/juobs/constants/juobs_const.jl")
am_ll = [m_N200[1], m_J303[1]]         ## array with light light effective masses for the two ensembles
beta = [3.55, 3.70]                    ## beta values 

m_ll = am_ll .*sqrt.(8 * t0.(beta))    ## masses in unit of sqrt(8 t_0). The t0 function returns the value of t0 for a given beta. It is defined in juobs_const
uwerr.(m_ll)                           ## run uwerr if you are interested in errors
println("The light-light mass is:\n", " N200: ", m_ll[1], " sqrt(8t0) \n J303: ", m_ll[2], " sqrt(8t0)" )

Alternatively, the t0 value ensemble per ensemble may be extracted from the ms.dat file rather than reading the value in juobs_coonst, taken from the scale setting paper. Let's see how to do that

In [ ]:
const path_N200r000_ms = "/Users/ale/Desktop/data/ms file/N200r000.ms.dat"
const path_N200r001_ms = "/Users/ale/Desktop/data/ms file/N200r001.ms.dat"

## two replicas N200
t1 = read_ms(path_N200r000_ms)                                  ## reads openQCD ms dat files and return a YData struct
t2 = read_ms(path_N200r001_ms)

t0_N200 = comp_t0([t1,t2], [20, 100], L=48, pl=true, npol=2)             ## computes t0  in a given plateau using the YData struct which contains the energy density of the YM action
                                                                        ##A polynomial interpolation in t is performed to find t0, where npol is the degree of the polynomial (linear fit by default)
t0_N200_rw = comp_t0([t1,t2] ,[20, 100], L=48, rw=[rwf_N200r000, rwf_N200r001])

uwerr(t0_N200)
uwerr(t0_N200_rw)

t0br = t0(beta[1])
uwerr(t0br)
print("\n t_0 values for the ensemble N200 : \n without rwf: ", t0_N200, " \n with rwf:    ", t0_N200_rw, "\n from Bruno:  ", t0br)


Mass derivative shifting applied to derived observables 

In [ ]:
sdr0_path = "/Users/ale/Desktop/data/pbp/N200r000.pbp.dat"  ## path to the pbp file containing the action derivatives
sdr1_path = "/Users/ale/Desktop/data/pbp/N200r001.pbp.dat"

## reading action derivatives with respect to the light and strange quark masses 
sdr0 = read_md(sdr0_path) ## vector containing action derivatives respect to the l and s quark masses
sdr1 = read_md(sdr1_path)

phi4 = 8*t0_N200_rw * ( m_N200[2]^2 + 1/2 * m_N200[1]^2 )            # phi4
phi4_ph = 1.11 #   uwreal([1.120, 0.024], "phi4")                    # phi4 physical

## phi4 derivatives w/ respect to m_l and m_s

phi_dml, phi_dms = md_sea(phi4, [sdr0, sdr1])
uwerr(phi_dml)
uwerr(phi_dms)
println( "d_ml = ", phi_dml, " d_ms = ", phi_dms)
dm = (phi4_ph- phi4) / ( 2 * phi_dml + phi_dms)
#uwerr(dm)
#println("dm is ", dm)

## meff derivatives w/ respect to m_l and m_s
m_dml, m_dms  = md_sea(m_N200[1], [sdr0, sdr1])

uwerr(m_dml)
uwerr(m_dms)
println( "m_dml = ", m_dml, " m_dms = ", m_dms)

## compute the shifted pion mass for N200
m_shifted = m_N200[1] + 2*value(dm)*value(m_dml) + value(dm)*value(m_dms) 
uwerr(m_shifted)
println("shifted mass           ", m_shifted, "\nunshifited pion mass   ", m_N200[1])


save/load data in BDIO format

In [ ]:
fb = BDIO_open("test.bdio", "w") ## open BDIO file in write mode

write_uwreal(m_N200[1], fb, 1)   ## writing m_N200
write_uwreal(m_J303[1], fb, 2)   ## writing m_J303
write_uwreal(f_N200[1], fb, 3)   ## writing f_N200 

BDIO_close!(fb)

In [ ]:
fb = BDIO_open("test.bdio", "r")     ## open BDIO file in read mode

BDIO_seek!(fb)                      ## move to first record
m1 = read_uwreal(fb)                ## reading m_N200 

BDIO_seek!(fb, 2)                   ## move forward  to third record
m2 = read_uwreal(fb)                ## reading m_J303

BDIO_seek!(fb, 2)                   ## move  forwward to fifth record
f1 = read_uwreal(fb)                ## reading f_N200

##check differences
println( " Better be zero: ", m1 - m_N200[1])
println( " Better be zero: ", m2 - m_J303[1])
println( " Better be zero: ", f1 - f_N200[1])
BDIO_close!(fb)

In [ ]:
##count the number of record 
fb = BDIO_open("test.bdio", "r")
BDIO_seek!(fb, 0)
count=0
while BDIO_seek!(fb)
     count += 1
end
println(count)
BDIO_close!(fb)

In [ ]:
## check the lenght of each record
fb = BDIO_open("test.bdio", "r")
count=0
BDIO_seek!(fb,0)
 while BDIO_seek!(fb)
     count += 1
     println("Record: ", count, " length: ", BDIO_get_len(fb), " bytes")
 end

BDIO_close!(fb)

Some issues with BDIO file: the function write_uwreal is suposed to create a new record each time you call it. In this example the function is called three times, hence we expect three different records. For some reason beyond my understanding, we actually have six different records. This is not particularly relevant, just keep in mind to jump 2 records each time you want to move further to the next variable stored, i.e. BDIO_seek!(fb,2)




A glampse to the  fitting routine with the LsqFit package  used for non linear fitting. This routine calls the ADerrors.fit_error to propagate the error in fit parameters. The method fit_error is agnostic about how you minimise the chisquare. 

In [ ]:
## we want to fit the ll correlator for N200, namely N200_pp[1]

@. model(x, p) = p[1]*exp(-p[2]*x) + p[3]*exp(-p[4]*x)   ## create a model function. @. notation applies the model to each element of the array x
#@. model(x,p) = p[1] + p[2] * exp(-(p[3]-p[1])*x)

source = getfield(N200_pp[1], :y0)                 ## source position
ydata = getfield(N200_pp[1], :obs)[source+6:end-5]  ## we consider the ll correlator at the right of the source
xdata = collect(1:length(ydata))

par = fit_routine(model, xdata, ydata, 4)          ## fitting routine that automatically prints usefull information

Plotting fit results 


In [ ]:
fig = figure()
    errorbar(xdata, value.(ydata), yerr=err.(ydata), fmt="*", ms=3)
    plot(xdata, model(xdata, value.(par)))
display(fig)

TUTORIAL ON THE GEVP 

If you followed my presentation at EuroPLEx talk, you may remember that I solved several GEVP's using different  local operators.
Here we focus on the following GEVP

  PP(t)      PP(t+3) 
  
  PP(t+3)     PP(t+6)

where we consider the PP correlator only with time shift.
The idea for this approach came from Lukas' talk at the Dublin Europlex meeting. You can find the slides at: 
https://indico.cern.ch/event/965890/contributions/4117130/attachments/2154411/3633546/Dublin2020_Varnhorst.pdf
Another reference is : arXiv:1010.0202 [hep-lat] 


In [ ]:
src = getfield(N200_pp[1], :y0) ## source position

## Heavy-Strange
hs11 = N200_pp[5].obs[src:end-7]              ## array of PP(t)
hs12 = N200_pp[5].obs[src+3:end-4]            ## array of PP(t+3)
hs22 = N200_pp[5].obs[src+6:end-1]            ## array of PP(t+6)

diag_hs = [hs11, hs22]                         ## correlators in the diagonal
subdiag_hs = [hs12]                            ## correlators in the upper diagonal elements
mat_hs = get_matrix(diag_hs, subdiag_hs)       ##  array of matrices for all timeslices

## Heavy-Heavy
hh11 = N200_pp[6].obs[src:end-7]              ## array of PP(t)
hh12 = N200_pp[6].obs[src+3:end-4]            ## array of PP(t+3)
hh22 = N200_pp[6].obs[src+6:end-1]            ## array of PP(t+6)

diag_hh = [hh11, hh22]                        ## correlators in the diagonal
subdiag_hh = [hh12]                           ## correlators in the upper diagonal elements
mat_hh = get_matrix(diag_hh, subdiag_hh)


In [ ]:
## now that we have the matrix, we are ready to  solve the gevp and compute the effective energies

eval_hs = getall_eigvals(mat_hs[1:end-1], 2) ## this method solve the gevp for all the matrices in the array mat_hs, taking t0=2
eval_hh = getall_eigvals(mat_hh[1:end-1], 2) ## this method solve the gevp for all the matrices in the array mat_hh, taking t0=2

en_hs = energies(eval_hs)                    ## this method computes the energies given the eigenvalues 
en_hh = energies(eval_hh)                    ## it returns a 1-d array with the length equal to the dimension of the matrix. Here it is 2.  

fig = figure()
    errorbar(collect(1:length(en_hs[1])), value.(en_hs[1]), yerr=err.(en_hs[1]), fmt="s", ms=3, mfc="none", mew=0.8,  elinewidth=0.5, mec="royalblue",ecolor="royalblue", capsize=2, label=L"$am_{D_s}$")
    errorbar(collect(1:length(en_hh[1])), value.(en_hh[1]), yerr=err.(en_hh[1]), fmt="s", ms=3, mfc="none", mew=0.8,  elinewidth=0.5, mec="tomato",ecolor="tomato", capsize=2, label=L"$am_{\eta_c}$")
    xlabel(L"$t$")
    ylabel(L"$am^{eff}$")
    title("Effective masses for N200")
    legend()
    ylim(0.62, 1)
display(fig)

In [ ]:

m_hs = plat_av(en_hs[1], [20,40])
m_hh = plat_av(en_hh[1], [18,45])
uwerr(m_hs)
uwerr(m_hh)

println("Heavy strange masses: \n standard: ", m_N200[5], "\n gevp :    ", m_hs)
println("\nHeavy heavy masses: \n standard: ", m_N200[6], "\n gevp :    ", m_hh)


Finally, let us explore how to extract the eigenvectors, usefull for the determination of decay constants. 
The procedure is similar to the eigenvalues, but here we will keep delta_t = t-t0 fixed, as proposed in arXiv:0902.1265v2 [hep-lat]

Also, they can be used to build the correlators that will have the largest overlap with the n^th state as reported in arXiv:1803.09673v1

In [ ]:
evec = getall_eigvecs(mat_hh[1:end-1], 3 ) # where \Delta t = t - t_0 = 3

# check eigenvectors
'''
aux = [uwdot(evec[i][:,1] , evec[i+1][:,1])^2 for i=1:length(evec)-1]
uwerr.(aux)
fig = figure()
    errorbar(collect(1:length(aux)), value.(aux), yerr=err.(aux), fmt="*")
    ylim(0.99995, 1.00005)
    
display(fig)
'''

So far you have seen two automatic method implemented in juobs_linalg, namely getall_eigvecs and getall_eigvals.
There might be times when you'd like to solve GEVP problems  for a single matrix, or with different settings rather than the one described for the above method.
If this is the case you have at your disposal a bunch of raw methods that act on a single matrix rather than a list of matrix.
They are :

            uweigvals(a)    # compute evals of a 
            uweigvals(a,b)  # compute generalised evals of a,b 
            
            uweigvecs(a)    # compute evecs of a 
            uweigvecs(a,b)  # compute generalised evecs of a,b 
            
            uweigen(a)      # compute evals and evecs of a 
            uweigen(a,b)    # compute generalised evals and evecs of a,b 
            
Here a,b are matrix of uwreal objects.
A more complete documentation can be found in the html page.

In [ ]:
a_mat = mat_hh[10]
b_mat = mat_hh[30]
println("This is a matrix of uwreal ")
a_mat

In [ ]:
uweigvals(a_mat)

In [ ]:
uweigvals(a_mat, b_mat)

And so on with uweigvecs() and uweigen()